In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from os import listdir
from os.path import isfile, join
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Using TensorFlow backend.


In [2]:
rm -rf /kaggle/working/

rm: cannot remove '/kaggle/working/': Device or resource busy


In [3]:
painting_path = '../input/painting-vs-photograph-classification-dataset/Raw Data/painting'

In [4]:
photo_path = '../input/painting-vs-photograph-classification-dataset/Raw Data/photos'

In [5]:

painting_images=[(f) for f in listdir(painting_path) if isfile(join(painting_path, f))]
photo_images = [(f) for f in listdir(photo_path) if isfile(join(photo_path, f))]



In [6]:
painting_df = pd.DataFrame()
painting_df['file_name']=[painting_path  +'/'+  str(x) for x in painting_images]

In [7]:
photo_df = pd.DataFrame()
photo_df['file_name']=[photo_path  +'/'+  str(x) for x in photo_images]

In [8]:
photo_df['category']= np.zeros(len(photo_df))
photo_df['category']= 0

In [9]:
painting_df['category']= np.zeros(len(painting_df))
painting_df['category'] = 1

In [10]:
df = painting_df.append(photo_df, ignore_index = True)

In [11]:
from sklearn.model_selection import train_test_split

import tensorflow as tf
print(tf.__version__)

2.2.0


In [12]:
image_augment = tf.keras.preprocessing.image.ImageDataGenerator(    
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

In [13]:
VALID_FRACTION = 0.2
TEST_FRACTION = 0.33
BATCH_SIZE = 256
EPOCHS = 50

IMAGE_WIDTH = IMAGE_HEIGHT = 256
IMAGE_CHANNELS = 3

In [14]:
train_df, test_df = train_test_split(df, test_size=TEST_FRACTION, random_state= 42)

In [15]:
train_df, valid_df = train_test_split(train_df, test_size=VALID_FRACTION, random_state= 42 )

In [16]:
from PIL import Image

In [17]:
from tqdm import tqdm

In [18]:
import cv2

In [19]:
train_df

,file_name,category
3988,../input/painting-vs-photograph-classification...,0
2933,../input/painting-vs-photograph-classification...,0
475,../input/painting-vs-photograph-classification...,1
3390,../input/painting-vs-photograph-classification...,0
2235,../input/painting-vs-photograph-classification...,0
...,...,...
2881,../input/painting-vs-photograph-classification...,0
4813,../input/painting-vs-photograph-classification...,0
1462,../input/painting-vs-photograph-classification...,0
4066,../input/painting-vs-photograph-classification...,0


In [20]:
save_iter = 0
new_arr =[]
for entry in tqdm(train_df.index):
    
    file_path = train_df.at[entry, 'file_name']
    category = train_df.at[entry, 'category']
    
    print((file_path))
    
    img =  cv2.imread(file_path)
    
    save_path = str(save_iter)+'.png'
    save_iter+=1
    
    cv2.imwrite(save_path, img)
    
    new_arr.append([save_path, category])
    
    
    image_single = img_to_array(img)
    image_single = expand_dims(image_single, 0)
    
    
    if(category == 1):
        
        for _ in range(   round(((len(train_df[train_df.category==0]))/(len(train_df[train_df.category==1]))))  *  4):
            
            it = image_augment.flow(image_single, batch_size = 1)
            batch = it.next()
            image_aug = batch[0].astype('float32')
            image_aug  = Image.fromarray(image_aug, 'RGB')
            save_path = str(save_iter)+'.png'
            image_aug.save(save_path)
            save_iter += 1
            
            new_arr.append([save_path, category])
    else:
        for _ in range(4):
            
            it = image_augment.flow(image_single, batch_size = 1)
            batch = it.next()
            image_aug = batch[0].astype('float32')
            image_aug  = Image.fromarray(image_aug, 'RGB')
            save_path = str(save_iter)+'.png'
            image_aug.save(save_path)
            save_iter += 1
            
            new_arr.append([save_path, category])
        

    

  0%|          | 0/2737 [00:00<?, ?it/s]

../input/painting-vs-photograph-classification-dataset/Raw Data/photos/e92272c233bb3ba5434926da5701c49bfe3580b7.jpg


  0%|          | 1/2737 [00:00<11:52,  3.84it/s]

../input/painting-vs-photograph-classification-dataset/Raw Data/photos/2bb9906c24b42da47a2ef4d113f208df28f01f2a.jpg


  0%|          | 2/2737 [00:00<11:35,  3.93it/s]

...

100%|█████████▉| 2735/2737 [17:30<00:00,  3.69it/s]

../input/painting-vs-photograph-classification-dataset/Raw Data/photos/7089c0ef9cd75f544abf7a009f5c0a5a92a99092.jpg
../input/painting-vs-photograph-classification-dataset/Raw Data/photos/edce9a5ab4b59124d1f9d417ab867a6151b54a26.jpg


100%|█████████▉| 2736/2737 [17:30<00:00,  4.11it/s]

../input/painting-vs-photograph-classification-dataset/Raw Data/painting/452420cf3f68158fec02f2ea74cc1c87d53a21d7.jpg


100%|██████████| 2737/2737 [17:31<00:00,  2.60it/s]
